# Optimizing SQL Server Backup Performance
## Using Data Science Techniques to Solve a Classic DBA Dilemma

There are several knobs we can turn to tune backup performance, but there isn't a great deal of guidance on what the best settings are for our environment, other than "try them and see!"  A big part of this is that the underlying hardware makes so much of a difference:  being I/O bound on backups means you want to configure things differently from if you are CPU-bound.  Also, if you are backing up a very busy system, you don't want to make the backup so well-tuned that it suddenly takes up 100% of your CPU.  This leads to a series of tradeoffs in configurable settings.

The most important of those settings are:
* Block Size -- The physical block size.  This really only matters for backup tapes and CD-ROMs but it is still settable.  Valid values:  { 0.5kb, 1kb, 2kb, 4kb, 8kb, 16kb, 32kb, 64kb }
* Max Transfer Size -- Maximum amount of data to be transferred per operation.  Valid values:  { 64kb, 128kb, 256kb, 512kb, 1mb, 2mb, 4mb }
* Buffer Count -- Number of buffers of size [MaxTransferSize] to be created.  Valid values:  { 1:N } but I probably wouldn't go above about 1024 without good reason, as with a MaxTransferSize of 4MB, that's up to 4GB of memory used for a single backup.
* File Count -- Tell SQL Server to stripe your backup across multiple files.  This is a nice way of getting extra throughput out of your backups.  Valid values:  { 1:N } but I probably wouldn't go above 10-12 without good reason.
* Compression -- Tell SQL Server whether or not you want to compress your backup.  This has a very minor cost of CPU but typically leads to **much** smaller backups, so my default is to say yes.  Valid values:  { TRUE, FALSE }

Armed with this knowledge, let's say you now want to go tune your systems.  Well, there are a **lot** of combinations.  Let's suppose that we go with the following options:
* Block Size:  { 0.5kb, 1kb, 2kb, 4kb, 8kb, 16kb, 32kb, 64kb }
* Max Transfer Size:  { 64kb, 128kb, 256kb, 512kb, 1mb, 2mb, 4mb }
* Buffer Count:  { 7, 15, 30, 60, 128, 256, 512, 1024 }
* File Count:  { 1..12 }
* Compression:  { TRUE, FALSE }

This gives us 10,752 separate options.  If your full backup averages 10 minutes, that's an expectation of 74.7 days straight of backups to try each of these options.  If you have a terabyte-sized backup which takes 90 minutes to complete, you'll get your answer in approximately 1.84 years.

But there's a not-so-secret weapon we can use:  sampling.  Without getting into the statistics of the problem, we can decide to take a random sample of the full set of options and perform an analysis on it.  With a reasonable-sized sample, we can get somewhere close to the actual population values in a fraction of the time.

My sample today is from two databases at six sizes.  I have one database called BAC which includes four separate versions:  the full 136 GB, 89.24 GB, 57.89 GB, and 31.73 GB, where the difference comes from dropping the largest tables one at a time.  In addition, I have two versions of the Stack Overflow database:  one from 2010 when it was 10 GB in size, and another from 2013 when it was 50 GB in size.

I built a Powershell script which builds a Cartesian product of my input arrays (that is, the parameters I laid out above) and runs the [dbatools](https://dbatools.io) cmdlet Backup-DbaDatabase.  I'm writing the output results to an output file.  Then, I manually added a header with the variable names to make it easier to import into R.  I'm sampling the Cartesian product, performing only about 1% of the total number of tests.  That's still a lot of tests, but it's a much more tractable problem:  it means taking about 100 database backups rather than 3000.

The Powershell code is available in the `SampleBackupOptions.ps1` script.

First, we will load the `tidyverse` package.  Then, we will load a package called `randomForest`.  This lets us use a random forest model to analyze our data.  We will load the `caret` package to help us partition training from test data.  Finally, the `evtree` pakage will let us build expected value trees using evolutionary learning (genetic algorithms).

In [ ]:
if (!require(tidyverse)) {
    install.packages("tidyverse", repos="http://cran.us.r-project.org")
    library(tidyverse)
}

if(!require(randomForest)) {
  install.packages("randomForest", repos = "http://cran.us.r-project.org")
  library(randomForest)
}

if(!require(caret)) {
  install.packages("caret", repos = "http://cran.us.r-project.org")
  library(caret)
}

if(!require(evtree)) {
    install.packages("evtree", repos = "http://cran.us.r-project.org")
    library(evtree)
}

I am using data from six databases of different sizes.  Each file has the same set of variables in the same order.

**NOTE** -- If you get an error when trying to load your own files, make sure that the file is in UTF-8 or ASCII format.  Powershell generates UCS-2 LE BOM files by default and R has trouble reading those.

In [ ]:
bac_31gb <- readr::read_delim("../data/BAC_31.73GB_PerfTest.csv", delim=",",
    col_names = c("BlockSize", "BufferCount", "MaxTransferSize", "FileCount", "CompressBackup", "Duration"),
    col_types = cols(
        BlockSize = col_integer(),
        BufferCount = col_integer(),
        MaxTransferSize = col_integer(),
        FileCount = col_integer(),
        CompressBackup = col_character(),
        Duration = col_integer()
    )
)

bac_58gb <- readr::read_delim("../data/BAC_57.89GB_PerfTest.csv", delim = ",",
  col_names = c("BlockSize", "BufferCount", "MaxTransferSize", "FileCount", "CompressBackup", "Duration"),
  col_types = cols(
      BlockSize = col_integer(),
      BufferCount = col_integer(),
      MaxTransferSize = col_integer(),
      FileCount = col_integer(),
	  CompressBackup = col_character(),
      Duration = col_integer()
))

bac_89gb <- readr::read_delim("../data/BAC_89.24GB_PerfTest.csv", delim = ",",
  col_names = c("BlockSize", "BufferCount", "MaxTransferSize", "FileCount", "CompressBackup", "Duration"),
  col_types = cols(
      BlockSize = col_integer(),
      BufferCount = col_integer(),
      MaxTransferSize = col_integer(),
      FileCount = col_integer(),
	  CompressBackup = col_character(),
      Duration = col_integer()
))

bac_136gb <- readr::read_delim("../data/BAC_136GB_PerfTest.csv", delim = ",",
  col_names = c("BlockSize", "BufferCount", "MaxTransferSize", "FileCount", "CompressBackup", "Duration"),
  col_types = cols(
      BlockSize = col_integer(),
      BufferCount = col_integer(),
      MaxTransferSize = col_integer(),
      FileCount = col_integer(),
	  CompressBackup = col_character(),
      Duration = col_integer()
))

so_10gb <- readr::read_delim("../data/SO2010_10GB_PerfTest.csv", delim = ",",
  col_names = c("BlockSize", "BufferCount", "MaxTransferSize", "FileCount", "CompressBackup", "Duration"),
  col_types = cols(
      BlockSize = col_integer(),
      BufferCount = col_integer(),
      MaxTransferSize = col_integer(),
      FileCount = col_integer(),
	  CompressBackup = col_character(),
      Duration = col_integer()
))

so_50gb <- readr::read_delim("../data/SO2013_50GB_PerfTest.csv", delim = ",",
  col_names = c("BlockSize", "BufferCount", "MaxTransferSize", "FileCount", "CompressBackup", "Duration"),
  col_types = cols(
      BlockSize = col_integer(),
      BufferCount = col_integer(),
      MaxTransferSize = col_integer(),
      FileCount = col_integer(),
	  CompressBackup = col_character(),
      Duration = col_integer()
))

wwi_3gb <- readr::read_delim("../data/WWI_3GB_PerfTest.csv", delim = ",",
  col_names = c("BlockSize", "BufferCount", "MaxTransferSize", "FileCount", "CompressBackup", "Duration"),
  col_types = cols(
      BlockSize = col_integer(),
      BufferCount = col_integer(),
      MaxTransferSize = col_integer(),
      FileCount = col_integer(),
	  CompressBackup = col_character(),
      Duration = col_integer()
))

I want to be able to combine the sets of data together and draw conclusions across the broader scope.  In order to differentiate the sets of data, I have added in a new variable, DatabaseSize.

In [ ]:
bac_31gb$DatabaseSize <- 31.73
bac_58gb$DatabaseSize <- 57.89
bac_89gb$DatabaseSize <- 89.24
bac_136gb$DatabaseSize <- 136.
so_10gb$DatabaseSize <- 10.
so_50gb$DatabaseSize <- 50.
wwi_3gb$DatabaseSize <- 3.7

In [ ]:
backupstats <- rbind(bac_31gb, bac_58gb, bac_89gb, bac_136gb, so_10gb, so_50gb, wwi_3gb)

In [ ]:
head(bac_31gb)

In [ ]:
head(backupstats)

In [ ]:
nrow(bac_31gb)

In [ ]:
nrow(backupstats)

To help interpret the results a bit easier, I'm converting block size to kilobytes.  This is a linear transformation of an independent variable, so this change does not affect the end results aside from scaling the betas.

In [ ]:
backupstats$BlockSizeKB <- backupstats$BlockSize / 1024.0
backupstats$BlockSize <- NULL

In [ ]:
head(backupstats)

We are also going to create a new measure, *MemoryUsageMB*.  The *MemoryUsageMB* measure combines the max transfer size with buffer count.  This is important because the **total amount of memory used** plays a role in backup duration, regardless of whether that memory comes in the form of more buffers or a larger buffer size.  For example, 7 buffers and a 128 KB max transfer size means that we will use 7 * 128KB = 896KB of memory for the backup itself.

In [ ]:
backupstats$MemoryUsageMB <- (backupstats$MaxTransferSize / (1024.0 * 1024.0)) * backupstats$BufferCount

In [ ]:
backupstats$BufferCount <- NULL
backupstats$MaxTransferSize <- NULL

In [ ]:
head(backupstats)

Here we create another measure, *SecPerGB*.  The *SecPerGB* measure gives us a measure of (inverse) throughput:  how many seconds does it take to transfer one GB of data to a backup?  This prevents database size from dominating our results.

In [ ]:
backupstats$SecPerGB <- backupstats$Duration / backupstats$DatabaseSize

In [ ]:
head(backupstats)

## Building Training and Test Data Sets

We are going to use the `caret` package to split out our data into separate training and test data sets.  This way, we can use the training data set to build a model for our given algorithm, and then our testing data set to give us an idea of how the model will perform on data it has not seen.

In [ ]:
set.seed(20191119)
randbackupstats <- backupstats[sample(nrow(backupstats)), ]

In [ ]:
trainIndex <- caret::createDataPartition(randbackupstats$SecPerGB, p = 0.7, list = FALSE, times = 1)
train_data <- randbackupstats[trainIndex,]
test_data <- randbackupstats[-trainIndex,]

In [ ]:
nrow(train_data)

In [ ]:
nrow(test_data)

In [ ]:
head(train_data)

This function calculates Root Mean Square Error, or RMSE.  RMSE is an important measurement because the output is in the same units as the measured values.  This means, for example, if RMSE is 8, for our case, that means that our prediction was typically within 8 seconds of the actual value.

In [ ]:
RMSE = function(m, o) {
    sqrt(mean((m-o)^2))
}

In [ ]:
# How we'll call the function
# RMSE(outcomes$Duration, outcomes$modelPred)

Here, we want to train a random forest model against `SecPerGB`, using all of our input features.  This will give us an idea of just how important each input is.

In [ ]:
model2 <- randomForest::randomForest(SecPerGB ~ BlockSizeKB + MemoryUsageMB + FileCount + DatabaseSize + CompressBackup,
                                    data = train_data,
                                    ntree = 2000,
                                    importance = TRUE)

When calculating importance, a value of `%IncMSE` less than 0 indicates that the feature is not at all important.  Larger positive numbers indicate relatively more important features.  In my case, the `CompressBackup` feature dominated, followed fairly closely by `DatabaseSize`.  This indicates that database size and whether I compress the backup are huge indicators of how long it takes the backup to complete.  File count and memory usage come next, and block size was actually negative, so it's not relevant at all in my consideration.  Your values may be different, so be sure to check them out.

In [ ]:
randomForest::importance(model2, scale=TRUE)

In [ ]:
model2

Now let's generate some predictions.  So far, we've seen how well the model does against the data it was trained with.  In practice, it won't be quite as effective as it first appears, as models can latch onto pecularities in the training dataset which don't exist outside of that dataset.

In [ ]:
modelPred2 <- predict(model2, test_data)

In [ ]:
outcomes2 <- cbind(test_data, as.data.frame(modelPred2))
outcomes2$PredictedDuration <- outcomes2$modelPred2 * outcomes2$DatabaseSize

In [ ]:
outcomes2$BlockSizeKB <- NULL

In [ ]:
head(outcomes2)

In [ ]:
RMSE(outcomes2$Duration, outcomes2$PredictedDuration)

In [ ]:
options(repr.plot.width=8, repr.plot.height=6)
ggplot(outcomes2, aes(x = Duration, y = PredictedDuration - Duration)) +
    geom_point()

In my case, RMSE ended up being approximately 9 seconds and we saw a lot of clustering around 0 for `PredictedDuration - Duration`.  On the whole, the model could be better, but should serve us reasonably well.

## EV Trees and Evolutionary Programming

Another approach we can take is to use one of the evolutionary techniques for developing a solution, such as genetic algorithms, genetic programming, or evolutionary programming.  This talk won't get into genetic algorithms directly, though if you are interested, you can review [my talk on the topic](https://csmore.info/on/genetics) and [my blog series on the topic](https://36chambers.wordpress.com/genetics-in-action/).  We'll use an evolutionary programming approach to solving the problem, though you won't need to know many details about how it works.

We will build an evolutionary tree which matches the regression tree from earlier.  Just like our prior demo, we will use the training data and compare against test data.

In [ ]:
ev <- evtree(SecPerGB ~ BlockSizeKB + MemoryUsageMB + FileCount + DatabaseSize + CompressBackup,
            data = train_data, minbucket = 10, maxdepth = 4)

In [ ]:
options(repr.plot.width=8, repr.plot.height=6)
plot(ev)

In [ ]:
ev

In my scenario, I ended up with the single best result being to compress the backups and then set `MemoryUsageMB` to at least 64MB.  You're looking for the **smallest** numbers, as we're measuring the number of seconds it takes to process one gigabyte of data in MDF/NDF files to a backup.  In your scenario, you might find a different set of rules--commonly, I see rules around the size of databases, where certain database sizes do better with different settings.  The key is that you want to narrow this down to as few branches as possible and pick the best outcomes given your tree elements.

Now let's generate some predictions and make sure that the model is close enough to what we saw last time around.

In [ ]:
test_data$PredSecPerGB <- predict(ev, test_data)

In [ ]:
test_data$PredDuration <- test_data$PredSecPerGB * test_data$DatabaseSize

In [ ]:
test_data %>%
    select(FileCount, DatabaseSize, MemoryUsageMB, SecPerGB, Duration, PredSecPerGB, PredDuration) %>%
    head()

In [ ]:
RMSE(test_data$Duration, test_data$PredDuration)

In [ ]:
ggplot(test_data, aes(x=Duration, y=PredDuration - Duration)) +
    geom_point()

It turned out that the model wasn't quite as accurate.  It still clustered fairly close to 0 seconds for `Predicted Duration - Duration` but the spread was wider and, more important, there were distinct linear perturbances.  This is a sign that we are missing at least one relevant feature which affected performance, but that said, the model was still close enough to our better random forest model that we can go ahead with it.  Also, the end result gives us something we can easily translate into rules, which we'll do next.